In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=40, 
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2  
)

In [3]:
data_directory = r"C:\Users\admin\Desktop\Projects\CS-Project\Tea-Leaf-Comprehensive-Anaysis\Data\Sickness" 

In [4]:
#MORE
train_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',
    subset='training' 
)

Found 926 images belonging to 8 classes.


In [5]:
Vald_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',
    subset='validation' 
)

Found 228 images belonging to 8 classes.


In [6]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,          
    input_shape=(224, 224, 3)   
)

In [7]:
base_model.trainable = True
for layer in base_model.layers[:-10]:  
    layer.trainable = False

In [8]:
model = tf.keras.Sequential()

In [9]:
model.add(base_model)

In [10]:
from tensorflow.keras import regularizers


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

In [11]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3, decay_steps=100000, decay_rate=0.96, staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,450,440 (139.05 MB)

 Trainable params: 17,328,392 (66.10 MB)

 Non-trainable params: 19,122,048 (72.94 MB)

In [13]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('Disease_classifier.keras', save_best_only=True)

In [14]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data = Vald_generator,
    callbacks= [early_stop, model_checkpoint]
)

Epoch 1/50


c:\Users\admin\Desktop\Projects\CS-Project\Tea-Leaf-Comprehensive-Anaysis\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 651s 17s/step - accuracy: 0.2354 - loss: 5.2200 - val_accuracy: 0.2982 - val_loss: 2.3014
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 538s 17s/step - accuracy: 0.3136 - loss: 2.2895 - val_accuracy: 0.2982 - val_loss: 2.2546
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 510s 16s/step - accuracy: 0.2919 - loss: 2.2453 - val_accuracy: 0.2982 - val_loss: 2.2125
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 528s 16s/step - accuracy: 0.2925 - loss: 2.2047 - val_accuracy: 0.2982 - val_loss: 2.1791
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 495s 16s/step - accuracy: 0.3127 - loss: 2.1646 - val_accuracy: 0.2982 - val_loss: 2.1514
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 593s 19s/step - accuracy: 0.2939 - loss: 2.1442 - val_accuracy: 0.2982 - val_loss: 2.1293
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 515s 16s/step - accuracy: 0.2950 - loss: 2.1261 - val_accuracy: 0.2982 - val_loss: 2.1122
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 539s 17s/step - accuracy: 0.2883 - loss: 2.1156 - val_accuracy: 0.2982 - val_

In [ ]:
val_loss, val_accuracy = model.evaluate(Vald_generator)
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [17]:
model.save('nitrogen_deficiency_classifier.keras')

In [ ]:
def predict_deficiency(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.  

    prediction = model.predict(img_array)
    categories = ['Healthy', 'algal leaf', 'Anthracnose', 'bird eye spot', 'brown blight', 'gray light', 'red leaf spot', 'white spot']



    predicted_class = np.argmax(prediction[0])
    
    print(f"Predicted Category: {categories[predicted_class]}")


predict_deficiency('Test8.jpg')  